In [1]:
from elasticsearch6 import Elasticsearch
import pandas as pd

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [3]:
host='localhost'
port = 9200
index =  "events-20220502"

In [4]:
client = Elasticsearch(f'{host}:{port}')

In [5]:
def query(perimeter, hostname='VM1', size=10000):
    return {
        "query": {
        "bool": {
          "must": [
            {
              "match": {
                "category": perimeter
              }
            },
            {
              "match": {
                "hostname": hostname
              }
            }
          ],
          "must_not": [],
          "should": []
        }
        },
        "from": 0,
        "size": size,
        "sort": [],
        "aggs": {}
        }


In [6]:
size = 10000
list_raw_data = []
list_raw_data.append(client.search(index=index, body=query('availability', size=size)))
list_raw_data.append(client.search(index=index, body=query('traceability', size=size)))
list_raw_data.append(client.search(index=index, body=query('integrity', size=size)))
list_raw_data.append(client.search(index=index, body=query('confidentiality', size=size)))

In [7]:
list_raw_data[3]['hits']['hits'][0]['_source']['details']

{'event': {'dataset': 'login',
  'category': ['authentication'],
  'type': ['start', 'authentication_failure'],
  'origin': '/host/var/log/btmp',
  'outcome': 'failure',
  'kind': 'event',
  'module': 'system',
  'action': 'user_login'},
 'service': {'type': 'system'},
 'ecs': {'version': '1.5.0'},
 'agent': {'version': '7.8.1',
  'type': 'auditbeat',
  'hostname': 'VM1',
  'ephemeral_id': 'a0d3acbd-22de-4ca3-960d-293f8b384e38',
  'id': '86dba4c3-a315-42df-b4d3-67e14c81f3d0',
  'name': 'VM1'},
 'related': {'user': ['root'], 'ip': ['0.0.0.0']},
 'message': 'Failed login by user root (UID: 0) on  (PID: 420576) from  (IP: 0.0.0.0)',
 'user': {'id': 0, 'name': 'root'},
 'process': {'pid': 420576},
 'endpoint': 'logstash',
 'customendpoint': True}

In [8]:
list_obs,list_package, list_tuple = [],[], []
for raw_data in list_raw_data:
    for results in raw_data['hits']['hits']:
        list_tuple.append((results['_source']['details']['event']['dataset'],results['_source']['details']['agent']['type']))
        list_obs.append(results['_source']['details']['event']['dataset'])
        list_package.append(results['_source']['details']['agent']['type'] )
        """
        if test ==  'uptime':
            print(i)
            pp.pprint(raw_data['hits']['hits'][i]['_source']['details'])
            break
        """




{'file', 'system.cpu', 'system.fsstat', 'beat.stats', 'mongodb.status', 'uptime', 'login', 'system.memory', 'system.network'}


{'heartbeat', 'auditbeat', 'metricbeat'}


{('system.memory', 'metricbeat'), ('system.cpu', 'metricbeat'), ('mongodb.status', 'metricbeat'), ('system.network', 'metricbeat'), ('file', 'auditbeat'), ('beat.stats', 'metricbeat'), ('system.fsstat', 'metricbeat'), ('login', 'auditbeat'), ('uptime', 'heartbeat')}


In [18]:

lp = list(set(list_package))
pp.pprint(lp)
print('\n')

lo = list(set(list_obs))
pp.pprint(lo)
print('\n')



lt = list(set(list_tuple))
lt

['heartbeat', 'auditbeat', 'metricbeat']


[ 'file',
  'system.cpu',
  'system.fsstat',
  'beat.stats',
  'mongodb.status',
  'uptime',
  'login',
  'system.memory',
  'system.network']




[('system.memory', 'metricbeat'),
 ('system.cpu', 'metricbeat'),
 ('mongodb.status', 'metricbeat'),
 ('system.network', 'metricbeat'),
 ('file', 'auditbeat'),
 ('beat.stats', 'metricbeat'),
 ('system.fsstat', 'metricbeat'),
 ('login', 'auditbeat'),
 ('uptime', 'heartbeat')]

In [25]:
for raw_data in list_raw_data:
    for results in raw_data['hits']['hits']:
         if results['_source']['details']['event']['dataset'] == 'uptime': # 'beat.stats': # 'file':
            pp.pprint(results['_source']['details'])
            break

{ 'agent': { 'ephemeral_id': 'a155fa20-d061-40e7-9cbb-80df4c50cb40',
             'hostname': 'test-conformite.netplus.priv',
             'id': '016bd43b-ef2a-47c2-8e55-e991357aec94',
             'name': 'test-conformite.netplus.priv',
             'type': 'heartbeat',
             'version': '7.9.3'},
  'customendpoint': True,
  'ecs': {'version': '1.5.0'},
  'endpoint': 'logstash',
  'event': {'dataset': 'uptime'},
  'hostname': 'test-conformite.netplus.priv',
  'icmp': {'requests': 1, 'rtt': {'us': 885}},
  'monitor': { 'check_group': '2c72c535-ec2e-11ec-91b0-005056a5b664',
               'duration': {'us': 1085},
               'id': 'dead-host-monitor-f5869335bca2e2ed',
               'ip': '192.168.122.3',
               'name': 'dead-host-monitor',
               'status': 'up',
               'timespan': { 'gte': '2022-06-14T22:06:10.000Z',
                             'lt': '2022-06-14T22:06:26.000Z'},
               'type': 'icmp'},
  'summary': {'down': 0, 'up': 1},
  'url

In [ ]:
for i in range(10000):
    try:
        pp.pprint(raw_data['hits']['hits'][i]['_source']['details']['mongodb']['status'])
        break
    except KeyError:
        pass

In [ ]:
for i in range(10000):
    try:
        pp.pprint(raw_data['hits']['hits'][i]['_source']['details']['uptime'])
        break
    except KeyError:
        pass